In [ ]:
from matplotlib import pyplot
from scipy.sparse import coo_matrix

import pyibm

%matplotlib inline

In [ ]:
pyibm.__version__

In [ ]:
def print_matrix_info(Op):
    print('Type: ', type(Op))
    print('Shape: ', Op.shape)
    print('Size: ', Op.data.size)
    print('Min/Max: ', Op.data.min(), Op.data.max())

In [ ]:
def plot_matrix(M, figsize=(6.0, 6.0), markersize=1):
    """Plot non-zero structure of an operator."""
    if not isinstance(M, coo_matrix):
        M = coo_matrix(M)
    pyplot.rc('font', family='serif', size=16)
    fig, ax = pyplot.subplots(figsize=figsize)
    ax.scatter(M.col, M.row, c='red', s=markersize, marker='s')
    ax.axis('scaled', adjustable='box')
    ax.set_xlim(0, M.shape[1] - 1)
    ax.set_ylim(0, M.shape[0] - 1)
    ax.invert_yaxis()
    ax.set_xticks([])
    ax.set_yticks([])
    return fig, ax

In [ ]:
grid = pyibm.GridBase()
grid

In [ ]:
config = dict(x=dict(start=0.0, end=1.0, num_cells=10),
              y=dict(start=2.0, end=3.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
config = dict(x=dict(start=0.0, end=1.0, num_cells=5),
              y=dict(start=2.0, end=3.0, num_cells=10),
              z=dict(start=4.0, end=5.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridc

In [ ]:
gridx = pyibm.GridFaceX(grid=grid)
gridx

In [ ]:
gridy = pyibm.GridFaceY(grid=grid)
gridy

In [ ]:
gridz = pyibm.GridFaceZ(grid=grid)
gridz

In [ ]:
ijk = (2, 3, 4)
for g in [grid, gridc, gridx, gridy, gridz]:
    for _ in range(10):
        ijk2 = g.ijk(g.idx(*ijk))
    assert ijk == ijk2

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy, gridz) 
print_matrix_info(GHat)
plot_matrix(GHat);

In [ ]:
MHat = pyibm.assemble_MHat(gridx, gridy, gridz)
print_matrix_info(MHat)
plot_matrix(MHat, figsize=(3.0, 3.0));

In [ ]:
G = MHat @ GHat
print_matrix_info(G)
plot_matrix(G);

In [ ]:
DHat = pyibm.assemble_DHat(gridc, gridx, gridy, gridz)
print_matrix_info(DHat)
plot_matrix(DHat);

In [ ]:
R = pyibm.assemble_R(gridx, gridy, gridz)
print_matrix_info(R)
plot_matrix(R, figsize=(3.0, 3.0));

In [ ]:
RInv = pyibm.assemble_RInv(R)
print_matrix_info(RInv)
plot_matrix(RInv, figsize=(3.0, 3.0));

In [ ]:
D = DHat @ RInv
print_matrix_info(D)
plot_matrix(D);

In [ ]:
# Check if divergence of the opposite of the transpose of the gradient.
Op = D + G.T
Op = Op.multiply(Op > 1e-12)
assert Op.nnz == 0